# Pre-Processing

In [1]:
import pandas as pd

In [7]:
df = pd.DataFrame(columns=[
    "day", "month", "year", "hour",
    "client", 
    "solar_power_cat", "solar_power_num", 
    # columns that will contain either (feat) TemperatureC,DewpointC,PressurehPa,WindDirectionDegrees,WindSpeedKMH,WindSpeedGustKMH,Humidity,HourlyPrecipMM,dailyrainMM,SolarRadiationWatts_m2
    # and significant negative contribution, negative contribution, positivie contribution and significant positive contribution
    "feat1", "feat2", "contri1", "contri2",
    # columns that will contain either none, no use, uncertain or use
    "air conditioner", "washing machine", "dishwasher", "water heater", "heater",
    "report"
])

df_meta = pd.read_csv("../GenerationCorpus/metadata.csv", index_col=0)

In [8]:
df_meta.head()

,date,client,solar_power_cat,solar_power_num,explanation,use_devices,uncertain_devices,nouse_devices,filename
0,"March 7th 2023, 10o'clock",Rafael,high,2.800,"#('solar radiation', 'positive contribution')",#air conditioner,#dishwasher,#,report_1_1.txt
1,"October 1st 2024, 7o'clock",Constantino,very low,0.457,"#('solar radiation', 'significant negative imp...",#,#,#water heater&dishwasher&washing machine,report_1_2.txt
2,"March 14th 2023, 15o'clock",Pedro,very high,3.920,"#('solar radiation', 'significant positive con...",#water heater&washing machine,#,#,report_1_3.txt
3,"April 7th 2024, 8o'clock",Rafael,medium,1.680,"#('hourly precipitation', 'negative impact')",#,#dishwasher,#washing machine&air conditioner,report_1_4.txt
4,"December 9th 2024, 6o'clock",Letícia,very low,0.543,"#('solar radiation', 'significant negative imp...",#,#,#air conditioner&washing machine,report_1_5.txt


In [9]:
df.head()

,day,month,year,hour,client,solar_power_cat,solar_power_num,feat1,feat2,contri1,contri2,air conditioner,washing machine,dishwasher,water heater,heater,report


In [17]:
import string

def seperate_date(line):
    components = line["date"].split()
    month = components[0]
    raw_day = components[1]
    if raw_day[1] in string.ascii_lowercase:
        day = raw_day[0]
    else:
        day = raw_day[0:1]

    year = components[2][0:4]

    raw_hour = components[3]
    if raw_hour[1] in string.ascii_lowercase:
        hour = raw_hour[0]
    else:
        hour = raw_hour[0:1]


def seperate_explanation(line):
    pass

def seperate_devices(line):
    pass

In [18]:
for i in df_meta.index:
    line = df_meta.iloc[i]
    seperate_date(line)

['March', '7th', '2023,', "10o'clock"]
['October', '1st', '2024,', "7o'clock"]
['March', '14th', '2023,', "15o'clock"]
['April', '7th', '2024,', "8o'clock"]
['December', '9th', '2024,', "6o'clock"]
['May', '28th', '2024,', "8o'clock"]
['April', '8th', '2023,', "7o'clock"]
['December', '27th', '2024,', "16o'clock"]
['June', '21st', '2024,', "11o'clock"]
['January', '26th', '2023,', "12o'clock"]
['January', '30th', '2024,', "8o'clock"]
['February', '27th', '2023,', "16o'clock"]
['May', '14th', '2023,', "14o'clock"]
['April', '2nd', '2024,', "18o'clock"]
['October', '10th', '2023,', "6o'clock"]
['February', '2nd', '2023,', "10o'clock"]
['February', '14th', '2023,', "20o'clock"]
['March', '25th', '2024,', "17o'clock"]
['December', '19th', '2023,', "9o'clock"]
['June', '30th', '2023,', "8o'clock"]
['April', '7th', '2024,', "11o'clock"]
['March', '20th', '2023,', "11o'clock"]
['February', '28th', '2024,', "12o'clock"]
['November', '6th', '2023,', "17o'clock"]
['January', '28th', '2023,', "14